In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import spacy
import flair

import sklearn

In [9]:
data = pd.read_csv('cleaned_data.csv')
raw_data = pd.read_csv('dataset.csv')

In [10]:
data.head()

,Date,Market,Keyword,CPC,Clicks,CTR,Impressions,Cost,AveragePosition,Year,Month
1,20120524,US-Market,agile management software,0.263034,1.326745,8.2,2.414973,1.405688,1.0,2012,5
9,20120524,US-Market,android app developers,-0.599462,1.367542,8.6,2.431364,1.184975,1.0,2012,5
13,20120524,US-Market,android application developer,0.028569,1.671358,8.6,2.737193,1.679155,1.0,2012,5
14,20120524,US-Market,android app developer,0.378512,2.168674,6.0,3.392521,2.283279,1.0,2012,5
17,20120524,US-Market,mobile applications developers,-0.810966,1.336260,8.3,2.414973,1.090258,1.0,2012,5


In [ ]:
## Generating word embeddings

nlp_model = spacy.load('en_vectors_web_lg')

nlp_model.vocab.vectors.shape

tokens = []
for keyword in data.Keyword.fillna(value=''):
    tokens.append(nlp_model(keyword))

data['keyword_length'] = pd.Series([len(keyword) for keyword in tokens])

data[data.keyword_length >= 7]

In [ ]:
## Trying out flair

from flair.embeddings import ELMoEmbeddings, DocumentRNNEmbeddings as DocEmbeddings

import flair
nlp_model = ELMoEmbeddings('original')

document_embeddings = DocEmbeddings([nlp_model], hidden_size=512, rnn_layers=2, rnn_type='LSTM')

from flair.models import TextClassifier
from flair.trainers import ModelTrainer

clf = TextClassifier()